## Task 1

In [1]:
import csv
import pandas as pd
from collections import defaultdict

def getdict():
    d = defaultdict(lambda: "NA")
    input_file = open("data_description.txt")
    for line in input_file:
        information = line.split('\t')
        # get rid of the information-less tuples
        if len(information) < 2 or information[0].strip().startswith('\n') or len(information[0])==0 or not information[0].startswith(' '):
            continue
        # build the dictionary from the tuples with data cleaning
        term = information[0].strip()
        desri = information[1].strip()
        d[term] = desri
    return d

term_dictionary = getdict()

# write dictionary to csv file
# output: two column data dictionary csv file
def write_dict_to_csv():
    with open('output/term_dict.csv', 'w') as f:
        f.write('Abbreviation,Description\n')
        for key in term_dictionary.keys():
            f.write("%s,%s\n" % (key, term_dictionary[key]))
            
write_dict_to_csv()

## Task 2

In [2]:
import bonobo
from bonobo.config import use_context_processor


def extract():
    yield from data.itertuples()


# map the terms(e.g. LandSlope) to full-term in dictionary
def transform(*args):
    # get the index for the columns that you want to look up
    args = list(args)

    # filter out the rows (yearbuilt < 1980)
    idx0 = category_dict['YearBuilt'] + 1
    if int(args[idx0]) < 1980:
        return None

    # TODO: you need to replace two or three abbreviations
    # plus 1 because the first element of args is the system idx
    idx1 = category_dict['LandSlope'] + 1
    idx2 = category_dict['LotShape'] + 1
    idx3 = category_dict['LandContour'] +1
    
    # replace the tuple values
    args[idx1] = term_dictionary[args[idx1]]
    args[idx2] = term_dictionary[args[idx2]]
    args[idx3] = term_dictionary[args[idx3]]
    return args[1:]


In [3]:
# function for writing to the files
def with_opened_file1(self, context):
    with open('output/output_lotinfo.csv', 'w+') as f1:
        wr1 = csv.writer(f1, delimiter=',')
        yield wr1
        
def with_opened_file2(self, context):       
    with open('output/output_homeinfo.csv', 'w+') as f2:
        wr2 = csv.writer(f2, delimiter=',')
        yield wr2
        
def with_opened_file3(self, context):
    with open('output/output_salesinfo.csv', 'w+') as f3:
        wr3 = csv.writer(f3, delimiter=',')
        yield wr3

In [4]:
# decorator is used here: every time we open the file, and append row to the existing rows, instead of overwriting it
@use_context_processor(with_opened_file1)
def write_repr_to_file1(wr1, *row):
    global first_line_written
    # uncomment to see why I need to replace "[" and "]"
    # f.write(repr(row))
    if row is None:
        return

    if not first_line_written:
        wr1.writerow([category_list[a] for a in lotinfo_idx])
        
    tuples_lotinfo = [row[0][a] for a in lotinfo_idx]
    wr1.writerow(tuples_lotinfo)

    first_line_written = True
    
@use_context_processor(with_opened_file2)
def write_repr_to_file2(wr2, *row):
    global first_line_written
    # uncomment to see why I need to replace "[" and "]"
    # f.write(repr(row))
    if row is None:
        return

    if not first_line_written:
        wr2.writerow([category_list[a] for a in homeinfo_idx])
        
    tuples_homeinfo = [row[0][a] for a in homeinfo_idx]
    wr2.writerow(tuples_homeinfo)
    
    first_line_written = True
    

@use_context_processor(with_opened_file3)
def write_repr_to_file3(wr3, *row):
    global first_line_written
    # uncomment to see why I need to replace "[" and "]"
    # f.write(repr(row))
    if row is None:
        return

    if not first_line_written:
        wr3.writerow([category_list[a] for a in salesinfo_idx])
        
    tuples_salesinfo = [row[0][a] for a in salesinfo_idx]
    wr3.writerow(tuples_salesinfo)    
    
    first_line_written = True
    

In [6]:
first_line_written = False

# prepare for the data
data = pd.read_csv('train.csv', encoding='ISO-8859-1')

# construct category dictionary (map the category to index)
category_dict = defaultdict()
category_list = data.columns.tolist()
for i in range(len(category_list)):
    category_dict[category_list[i]] = i
print(category_dict)


# divide the table into 3 sub-tables
# TODO: The index needs to be changed according to the requirements
lotinfo_idx = [0,3,4,5,6,7,8,9,10,11]
homeinfo_idx = [0,15,16,17,18,19,20,21,22,23,24,25,26,27,28,
                    29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,
                    45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,
                    61,62,63,64,65,66,67,68,69,70,71,72,73,74]
salesinfo_idx = [0,75,76,77,78,79,80]
# build Bonobo pipeline
graph1 = bonobo.Graph()
graph1.add_chain(
    extract,
    # the transform step will replace the abbr. with its full description
    transform,
    bonobo.Limit(100),
    write_repr_to_file1,
)
bonobo.run(graph1)

first_line_written = False

graph2 = bonobo.Graph()
graph2.add_chain(
    extract,
    # the transform step will replace the abbr. with its full description
    transform,
    bonobo.Limit(100),
    write_repr_to_file2,
)
bonobo.run(graph2)

first_line_written = False

graph3 = bonobo.Graph()
graph3.add_chain(
    extract,
    # the transform step will replace the abbr. with its full description
    transform,
    bonobo.Limit(100),
    write_repr_to_file3,
)
bonobo.run(graph3)


defaultdict(None, {'Id': 0, 'MSSubClass': 1, 'MSZoning': 2, 'LotFrontage': 3, 'LotArea': 4, 'Street': 5, 'Alley': 6, 'LotShape': 7, 'LandContour': 8, 'Utilities': 9, 'LotConfig': 10, 'LandSlope': 11, 'Neighborhood': 12, 'Condition1': 13, 'Condition2': 14, 'BldgType': 15, 'HouseStyle': 16, 'OverallQual': 17, 'OverallCond': 18, 'YearBuilt': 19, 'YearRemodAdd': 20, 'RoofStyle': 21, 'RoofMatl': 22, 'Exterior1st': 23, 'Exterior2nd': 24, 'MasVnrType': 25, 'MasVnrArea': 26, 'ExterQual': 27, 'ExterCond': 28, 'Foundation': 29, 'BsmtQual': 30, 'BsmtCond': 31, 'BsmtExposure': 32, 'BsmtFinType1': 33, 'BsmtFinSF1': 34, 'BsmtFinType2': 35, 'BsmtFinSF2': 36, 'BsmtUnfSF': 37, 'TotalBsmtSF': 38, 'Heating': 39, 'HeatingQC': 40, 'CentralAir': 41, 'Electrical': 42, '1stFlrSF': 43, '2ndFlrSF': 44, 'LowQualFinSF': 45, 'GrLivArea': 46, 'BsmtFullBath': 47, 'BsmtHalfBath': 48, 'FullBath': 49, 'HalfBath': 50, 'BedroomAbvGr': 51, 'KitchenAbvGr': 52, 'KitchenQual': 53, 'TotRmsAbvGrd': 54, 'Functional': 55, 'Firep

BonoboWidget()

BonoboWidget()

BonoboWidget()